In [ ]:
project_id = input("Please enter the project you'd like to work in\n")

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
proceed = True
while proceed:
    origin_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone from:\n")
    target_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone to:\n")
    dataset_id = f'{project_id}.{origin_schema}'

    tables = client.list_tables(dataset_id)  # Make an API request.

    print("Tables contained in '{}':".format(dataset_id))
    for table in tables:
        print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        if table.table_type == 'VIEW':
            clone_table_statement=f"""
            CREATE OR REPLACE VIEW
              {project_id}.{target_schema}.{table.table_id}
              AS SELECT * FROM {project_id}.{origin_schema}.{table.table_id}
            """
        else:
            clone_table_statement=f"""
            CREATE OR REPLACE TABLE
              {project_id}.{target_schema}.{table.table_id}
              CLONE {project_id}.{origin_schema}.{table.table_id}
            """
        client.query(clone_table_statement).result()
        print(f"Cloned to {project_id}.{target_schema}.{table.table_id}")
    proceed = input("\nIf you'd like to clone another schema please enter 'yes'; only the full word yes will be accepted")=='yes'